# SQL Lab

Congratulations! We've been hired as analysts at Classic Model Car Company, and you've been handed keys to the Data Warehouse.  They want you to hit the ground running with analysis! We have lots of questions to answer.  First, let's connect to the database.

1. Connect via MySQL Credentials
  - hostname: `hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com`
  - username: `hbs-student`
  - password: (distributed via slack)
  - port: 3306 (default)
  - database: `classicmodels`

2. Take a moment to study the schema, noting the keys. Browse the data in each relation if you wish. Answer the following queries. I've given you the expected results below each query.

3. Answer the following questions! Do them in your MySQL client first, most likely, then bring them into Pandas here in this notebook.

In [1]:
!pip install mysql-connector-python

In [2]:
import pandas as pd
import mysql.connector

con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='classicmodels')
cur = con.cursor()

1. List all customer names and their addresses who don't have a specified sales rep. Order by customer name.

In [5]:
cur.execute('''SELECT contactFirstName, contactLastName, addressLine1, addressLine2
               FROM classicmodels.customers
               WHERE salesRepEmployeeNumber IS NULL
               ORDER BY 2
''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,contactFirstName,contactLastName,addressLine1,addressLine2
0,Raanan,"Altagar,G M",3 Hagalim Blv.,None
1,Mel,Andersen,Obere Str. 57,None
2,Carmen,Anton,"c/ Gobelas, 19-1 Urb. La Florida",None
3,Alejandra,Camino,"Gran Vía, 1",None
4,Philip,Cramer,Maubelstr. 90,None
5,Isabel,de Castro,Estrada da saúde n. 58,None
6,Alexander,Feuer,Heerstr. 22,None
7,Peter,Franken,Berliner Platz 43,None
8,Ed,Harrison,Rte des Arsenaux 41,None
9,Karin,Josephs,Luisenstr. 48,None


2. List the names of all customers that have a creditLimit between 100,000 and 200,000 inclusively. Order by customer name.

In [9]:
cur.execute('''SELECT contactLastName, contactFirstName
               FROM customers
               WHERE 100000 < creditLimit < 200000
               ORDER BY contactLastName
''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,contactLastName,contactFirstName
0,Accorti,Paolo
1,"Altagar,G M",Raanan
2,Andersen,Mel
3,Anton,Carmen
4,Ashworth,Rachel
...,...,...
117,Yoshido,Juri
118,Young,Jeff
119,Young,Julie
120,Young,Mary


CHECK
3. List all employees who have the initials of M.P. or P.M., by name. Order by last name.

In [12]:
cur.execute('''SELECT contactLastName, contactLastName
               FROM customers
               WHERE (contactLastName = 'M*' AND contactFirstName = 'P*') OR (contactLastName = 'P*' AND contactFirstName = 'M*')
               ORDER BY 1
''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

4. List all the products' productNames purchased by Mini Wheels Co. Order the list by product line.

In [17]:
cur.execute('''SELECT productName, productLine
               FROM products
               JOIN orderdetails
               USING(productCode)
               JOIN orders
               USING(orderNumber)
               JOIN customers
               USING(customerNumber)
               GROUP BY customerName
               WHERE customerName = 'Mini Wheels Co.'
               ORDER BY productLine
''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'WHERE customerName = Mini Wheels Co
               ORDER BY productLine' at line 10

5. List the customers by customerName whose contact person shares either a first name or last name with any of the company's employees Order by customerName

In [18]:
cur.execute('''SELECT customerName
               FROM customers
               WHERE contactLastName IN (SELECT lastName
                                         FROM employees)
               WHERE contactFirstName IN (SELECT firstName
                                          FROM employees)
               ORDER BY customerName
''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'WHERE contactFirstName IN (SELECT firstName
                                    ' at line 5

7. List the customers (by name) that are based in the country which is first in the lexicographic order of customer countries. Order by customerName

In [20]:
cur.execute('''SELECT customerName
               FROM customers
               WHERE country IS (SELECT country
                                 FROM customers
                                 ORDER BY 1
                                 LIMIT 1)
''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '(SELECT country
                                 FROM customers
                ' at line 3

8. Identify the product(s) by productCode and productName that the company has the least stock of. Order by product code.

In [21]:
cur.execute('''SELECT productCode, productName
               FROM products
               WHERE quantityInStock = MIN(quantityInStock) 
               ORDER BY productCode
''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

DatabaseError: 1111 (HY000): Invalid use of group function

9. Identify the offices (by city) that have the least employees and the most employees (answer should include city, numEmps)

In [ ]:
cur.execute('''SELECT MAX()
               FROM offices
               

''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

10. For all Sales Reps list their names (first and last) and the name of the office they work out of and the number of clients they represent. Order by reverse order of numClients

11. List the customer(s) and their total payments (answer should include customerNumber, customerName, totalPayment where totalPayment is the total amount of dollars the customer has paid). Order by totalPayment.

12. List the product number, product name, and customer name such that the customer has never ordered that product. Order by customerName, productCode.

13. List the profit per product that the company would make if it were able to sell all of that product at MSRP (answer should include productName, profit). Order by profit.

14. List the average order size for each customer (answer should include customerName, avgQuantity) Order by customer name.